## 1. Set up, data ingestion and conversion of excel table to CSV and a list of data frames

### 1.1 Import dependencies

In [1]:
import pandas as pd
# !pip install openpyxl
import openpyxl
from sqlalchemy import create_engine
import sqlite3 as sql
import pandas as pd


### 1.2 Retrieve data from excel files

In [2]:
excel_file = 'Data_Tables_LGA_Criminal_Incidents_Year_Ending_March_2022.xlsx'
all_sheets = pd.read_excel(excel_file, sheet_name=None)
sheets = all_sheets.keys()

for sheet_name in sheets:
    sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
    sheet.to_csv("%s.csv" % sheet_name, index=False)

Contents
Table 01

Criminal incidents and rate per 100,000 population by police region and local government area - April 2012 to March 2022
Table 02

Criminal incidents and rate per 100,000 population by principal offence, local government area and police service area - April 2012 to March 2022
Table 03

Criminal incidents by principal offence, local government area and postcode or suburb/town - April 2012 to March 2022
Table 04

Criminal incidents by location type and local government area - April 2012 to March 2022
Table 05

Criminal incidents by charge status and local government area - April 2012 to March 2022
Data exclusions
Table 01

excludes criminal incidents where the geographic location is unknown. For further information of these geographic locations please refer to the Explanatory Notes and Glossary on the website.
Tables 02 - 05

excludes criminal incidents at Justice institutions and immigration facilities, Unincorporated Victoria and where the geographic location is unknown. For further information of these geographic locations please refer to the Explanatory Notes and Glossary on the website.
Notes
Recorded crime statistics are based on data extracted by Victoria police on the 18th day after the reference period, and are subject to movement between releases. For more information about how statistics are compiled, refer to the Explanatory notes on the CSA website.
In order to maintain confidentiality, sensitive offence counts for subdivisions 'A10 Homicide and related offences' and 'A30 Sexual offences' with a value of 3 or less are given a value of 2 to calculate totals. Rates are based on populations provided by the Australian Bureau of Statistics (ABS). The most recent year of data was not available from the ABS in time for it to be used to calculate current year rates. The CSA uses estimates created by the Victorian State Government 'Victoria in Futures' report.
This work is licenced under a Creative Commons Attribution 4.0 International License. When reporting this data you must attribute the Crime Statistics Agency (or CSA) as the source.

### 1.3 Loop thru excel file to convert various worksheets to csv files

In [3]:
#Create list of dataframes from csv files
file_list = []
for i in range(1,7):
    file_list.append(f"Table 0{i}.csv")

df_list = [pd.read_csv(file) for file in file_list]
file_list

['Table 01.csv',
 'Table 02.csv',
 'Table 03.csv',
 'Table 04.csv',
 'Table 05.csv',
 'Table 06.csv']

## 2. Data cleaning and organization

### 2.1 Create a data frame that matches Police Regions with LGAs. 

#### Table 01.csv shows the number of incidents by Police Region and LGA – it is is the only table that shows the Police Region.
#### The Police Region is required for filtering within the data visualisation 


In [4]:
# 2.1.1 create data frame
region_lga_df = pd.read_csv("Table 01.csv")
#region_lga_df

In [5]:
# 2.1.2 strip out blanks from Local Government Area(LGA) and drop duplcates to 
# create a datafream that matched Police Regions to LGA
region_lga_df["Local Government Area"] = region_lga_df["Local Government Area"].str.strip()
region_lga_map_df =region_lga_df[["Local Government Area","Police Region"]].drop_duplicates().reset_index(drop=True)
# region_lga_map_df.info()
# region_lga_map_df.head()


### 2.2 Create and clean LGA_Offence_df_2019_2022 from Table 02 which is more granular in terms of both geographical areas and offence types.
#### Specifically, Table 02.csv holds data at Police Area, LGA, postcode and town/suburb areas as well as all three Offence categories form 2013 to 2022. It is missing the Police Region Area level 

In [6]:
# 2.2.1 Exract rows that are greater than year 2018 (use 4 years data, 2019-2022) 
# and drop column 'Year ending' as the crime tables are created on March (no date details) every year.
lga_offence_2019_2022_df = df_list[1].loc[df_list[1].Year > 2018].drop ('Year ending', axis=1)
# lga_offence_2019_2022_df.info()

In [13]:
# 2.2.2. Join Police Region onto the lga_offence_2019_2022_df
region_psa_lga_offence_2019_2022_df=pd.merge(lga_offence_2019_2022_df, region_lga_df [["Police Region","Local Government Area"]],
       on="Local Government Area")

offence_2019_2022_df = region_psa_lga_offence_2019_2022_df.drop_duplicates()
# offence_2019_2022_df.head()

## 3. Transform Data for Analysis 1 - overview of Offence type by Offence Division by Year 

#### 3.1 Create a total number of incidents variable to enable the total number of incidents by all LGAs to be shown on the 'Number of Incidents by Offence Division' chart js under an 'All' category

In [7]:
# create and save groupby offence division - DONE - See below  
# then sum up incidents recorded record output to a variable - DONE - see below 
# pd.concat to filter_df save as json file  THIS IS MY QUESTION!!!
# n_incidents_OD_df = offence_2019_2022_df.groupby("Offence Division")["Incidents Recorded"].sum()
# n_incidents_OD_df.head()

#### 3.2 Create a Offence Division Summary df to enable a high-level view of the number of Offences by Offence Category by Year ( analysis 1)



In [8]:
offence_division_summary_df = df_list[5]
#    .drop('Year ending',axis=1)
offence_division_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 8 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Year                                  320 non-null    int64  
 1   Local Government Area                 320 non-null    object 
 2   A Crimes against the person           320 non-null    int64  
 3   B Property and deception offences     320 non-null    int64  
 4   C Drug offences                       318 non-null    float64
 5   D Public order and security offences  320 non-null    int64  
 6   E Justice procedures offences         320 non-null    int64  
 7   F Other offences                      311 non-null    float64
dtypes: float64(2), int64(5), object(1)
memory usage: 20.1+ KB


In [9]:
# 3.2.1  Join Police Region onto the offence_division_summary_df
offence_division_region_df=pd.merge(offence_division_summary_df, region_lga_df [["Police Region","Local Government Area"]],
       on="Local Government Area")

offence_division_region_df= offence_division_region_df.drop_duplicates()
# offence_division_region_df

#####My first attempt below ... that did not work - 

# here are othere oiptions - do ou think they will work? 

# Use DataFrame.sum() method
df2 = df['math'].sum()

# Using DataFrame.sum() method 
df2 = sum(df['math'])

# Use DataFrame.loc[] and pandas.Series() to get total of columns
df.loc['Total'] = pd.Series(df['math'].sum(), index = ['math'])

# Get total of columns using DataFrame.loc[] method
df.loc['Total'] = df["math"].sum()

# Use DataFrame.loc[] & DataFrame.sum() Method
df.loc["Total", "math"] = df.math.sum()

# Use DataFrame.at[] method to get total of columns
df.at['Total', "math"] = df["math"].sum()

# Use DataFrame.append() method
df2 = df.append(pd.DataFrame(df.math.sum(), index = ["Total"], columns=[ "math"]))
print(df2)


In [10]:
# Join n_incidents_OD  nto the lga_offence_2019_2022_df
# attempt 1
# offence_division_summary_df =pd.merge(lga_offence_2019_2022_df, n_incidents_OD_df[["offence Division"]],

       # on="Local Government Area").set_index('Year')

# attempt 2 - outcome 
# offence_division_summary_df3 = pd.concat([offence_division_summary_df, n_incidents_OD_df], ignore_index=True, sort=False)
# offence_division_summary_df3

#attempt 3 
# data =offence_division_summary_df, n_incidents_OD_df
# offence_division_summary_df4 = pd.concat (data)
# offence_division_summary_df4 

#attempt 4
# offence_division_summary_df4 = pd.concat([offence_division_summary_df, n_incidents_OD_df], axis=1, join='inner')
# offence_division_summary_df4 

#attempt 5 - nearly there need to add year and change Nan in "Year" to the relevant year and NaN in "Local Government Area" tp "All"
# offence_division_summary_df4 = offence_division_summary_df.append(n_incidents_OD_df, ignore_index=True)

# offence_division_summary_df4['Local Government Area'] = offence_division_summary_df4['Local Government Area'].replace(['NaN'],'All')
# # offence_division_summary_df4.info()
# offence_division_summary_df4

In [9]:
# offence_division_summary_df=pd.merge(offence_division_summary_df, region_lga_df [["Police Region","Local Government Area"]],
#        on="Local Government Area")

# offence_division_summary_2019_2022_df = offence_division_summary_df.drop_duplicates()
# offence_division_summary_2019_2022_df.head()

In [10]:
# 3.2.2.Create a list of Police Regions to enable filtering by Police Region
policeRegions = offence_division_region_df['Police Region'].unique().tolist()
policeRegions

['2 Eastern', '4 Western', '1 North West Metro', '3 Southern Metro']

In [11]:
# 3.2.3.Create a list of LGAs to enable filtering by LGA

lgas = offence_division_summary_df['Local Government Area'].unique().tolist()
# lgas

### 3.3 Save as CSV and export as JSON file

In [12]:


import json

payload = {}

for lga in offence_division_summary_df['Local Government Area'].unique().tolist():
    df = offence_division_summary_df.loc[offence_division_summary_df['Local Government Area'] == lga].groupby\
        (['Local Government Area','Year']).agg(sum).reset_index().drop('Local Government Area',axis =1).set_index('Year')
    df
    dataStackedBar = {'labels': df.index.tolist(),'datasets':[]}

    colors = ['#cbd4c2ff','#dbebc0ff','#c3b299ff','#815355ff','#523249ff','#000000']
    color = 0
    for col in df.columns:
        dataStackedBar['datasets'].append({
            'label': col,
            'data': df[col].values.tolist(),
            'backgroundColor': colors[color]
        })
        color += 1
    payload[lga] = dataStackedBar

with open('bardata.json','w') as output:
    json.dump(payload,output)

## 4. Create 2022 data frame for Analysis 3 - Crime patterns for 2022 by Offence Types

In [13]:
lga_offence_2022_df = df_list[1].loc[df_list[1].Year > 2021].drop ('Year ending', axis=1)
# lga_offence_2022_df
lga_offence_2022_df2=pd.merge(lga_offence_2022_df, region_lga_df [["Police Region","Local Government Area"]],
       on="Local Government Area")

offence_2022_df=lga_offence_2022_df2.drop_duplicates()
# offence_2022_df
# offence_2022_df= offence_2019_2022_df[offence_2019_2022_df].Year > 2021

## 5. Export dataframes to csv files for checking data quality

In [14]:
offence_division_summary_df.to_csv('Offence_division_summary.csv',index=0)
crime_offence_division_summary = pd.read_csv('Offence_division_summary.csv')

In [44]:
offence_2019_2022_df.to_csv('crime_data_2019_2022.csv',index=0)
crime_offence_2019_2022 = pd.read_csv('crime_data_2019_2022.csv')

In [15]:
offence_2022_df.to_csv('crime_data_2022.csv',index=0)
crime_offence_2022_ = pd.read_csv('crime_data_2022.csv')

## 6. Connect to Data Base

In [21]:
conn = sql.connect('crime.db')
crime.to_sql('crime_offence_2019_2022', conn)

NameError: name 'crime' is not defined

In [17]:
all_df = pd.read_sql('SELECT * FROM crime_offence_2019_2022', conn)

In [18]:
filter_df = all_df.groupby(["Year","Offence Division","Offence Subdivision"])["Incidents Recorded"].sum()
filter_df = filter_df.reset_index()
filter_df

KeyError: 'Offence Division'

In [19]:
all_df.head()

,index,Year,Local Government Area,A Crimes against the person,B Property and deception offences,C Drug offences,D Public order and security offences,E Justice procedures offences,F Other offences,Police Region
0,0,2019,Alpine,114,161,26.0,48,50,NaN,2 Eastern
1,1,2020,Alpine,92,220,24.0,47,52,4.0,2 Eastern
2,2,2021,Alpine,103,138,21.0,37,62,39.0,2 Eastern
3,3,2022,Alpine,86,149,16.0,33,61,25.0,2 Eastern
4,4,2019,Ararat,223,376,65.0,81,165,2.0,4 Western


## 4.  Transform Data for Analysis 2: Stacked Circles by Year, Offence Division and Offence Subdivision

#### 4.1 Create flare for stacked Circles by Year, Offence Division and Offence Subdivision

In [20]:
import json

#https://stackoverflow.com/questions/59946453/creating-a-flare-json-to-be-used-in-d3-from-pandas-dataframe
#initialise a flare dictionary
flare = {"name": "flare", "children": []}

# iterate through dataframe values
for row in filter_df.values:
    level0 = row[0]
    level1 = row[1]
    level2 = row[2]
    value = row[3]
    
    # create a dictionary with all the row data
    d = {'name': level0,
          'children': [{'name': level1,
                        'children': [{'name': level2,
                                      'value': value}]}]}
    # initialize key lists
    key0 = []
    key1 = []

    # iterate through first level node names
    for i in flare['children']:
        key0.append(i['name'])

        # iterate through next level node names
        key1 = []
        for _, v in i.items():
            if isinstance(v, list):
                for x in v:
                    key1.append(x['name'])

    # add the full row of data if the root is not in key0
    if level0 not in key0:
        d = {'name': level0,
              'children': [{'name': level1,
                            'children': [{'name': level2,
                                          'value': value}]}]}
        flare['children'].append(d)

    elif level1 not in key1:

        # if the root exists, then append only the next level children

        d = {'name': level1,
              'children': [{'name': level2,
                            'value': value}]}

        flare['children'][key0.index(level0)]['children'].append(d)

    else:

        # if the root exists, then only append the next level children
        
        d = {'name': level2,
             'value': value}

        flare['children'][key0.index(level0)]['children'][key1.index(level1)]['children'].append(d)

# uncomment next three lines to save as json file
# save to some file
with open('flare.json', 'w') as outfile:
    json.dump(flare, outfile)

print(json.dumps(flare, indent=2))

NameError: name 'filter_df' is not defined